In [38]:
import sys
sys.path.append('../../scripts/')
from query import *
import pandas as pd


In [39]:
df_HWO = pd.read_excel('../../data/HWO_target_list_164.xlsx')
df_HWO

df_HWO['hd_name'] = df_HWO['hd_name'].fillna('').str.replace(r'\s+', '', regex=True)

filename = '../../data/HWO_target_list_164_update.xlsx'
df_HWO.to_excel(filename, index=False)

df_HWO

,hip_name,hip_compname,hd_name,hr_name,gj_name,constellation,hostname,sy_dist,ra,dec,sy_plx,sy_vmag,st_spectype,st_teff,st_tefferr,st_lum,st_rad,st_mass
0,HIP 42438,NaN,HD72905,HR 3391,GJ 311,3 UMa,3 Ursae Majoris A,144388,1297987692,650209064.0,692576,5630,G0.5V Fe-0.5,5893,14.0,-0.011800,946,01:08:00
1,HIP 44897,NaN,HD78366,HR 3625,GJ 334.2,NaN,HD 78366,189498,1372127933,338822184.0,527709,5962,G0IV-V,5992,20.0,0.714583,1045,01:08:00
2,HIP 45038,A,HD78154A,HR 3616 A,GJ 335A,13 UMa A,13 Ursae Majoris A,205169,1375981111,671340172.0,487404,4809,F7V,6325,42.0,4.233333,1680,01:21:00
3,HIP 48113,NaN,HD84737,HR 3881,GJ 368,NaN,HD 84737,188226,1471473806,460210074.0,531277,5086,G0(V),5893,13.0,3.100000,1604,01:08:00
4,HIP 51459,A,HD90839,HR 4112 A,GJ 395,36 UMa A,36 Ursae Majoris A,129452,1576565803,559805388.0,772485,4820,F8V,6164,21.0,1.427083,1111,01:18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,HIP 23693,NaN,HD33262A,HR 1674 A,GJ 189,zet Dor A,Zeta Doradus A,116927,763777320,-574727047.0,855233,4701,F9V Fe-0.5,6158,23.0,1.154167,1064,01:14:00
160,HIP 69965,A,HD125276A,HR 5356 A,GJ 9476A,NaN,HD 125276 A,179888,2147537320,-258154250.0,555900,5872,F9V Fe-1.5 CH-0.7,6120,41.0,0.570833,978,01:11:00
161,HIP 910,NaN,HD693,HR 33,GJ 10,6 Cet,6 Ceti,188861,28160752,-154679779.0,529489,4895,F8V Fe-0.8 CH-0.5,6190,13.0,3.310417,1505,01:18:00
162,HIP 112447,A,HD215648A,HR 8665,GJ 872A,xi Peg A,Xi Pegasi A,161509,3416732494,121728858.0,619161,4200,F6V,6193,23.0,4.566667,1852,01:25:00


In [40]:
merged_df = pd.read_excel('../../results/stars_without_bright_neighbors.xlsx', dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str})
merged_df[['HD Number 1', 'HD Number 2']] = merged_df['HD Number'].str.split(', ', expand=True, n=1)
# Clean up 'HD Number 1' and 'HD Number 2' by removing extra spaces after 'HD'
merged_df['HD Number 1'] = merged_df['HD Number 1'].str.replace(r'HD\s+', 'HD', regex=True)
merged_df['HD Number 2'] = merged_df['HD Number 2'].fillna('').str.replace(r'HD\s+', 'HD', regex=True)

merged_df = merged_df[merged_df['HZ Detection Limit [M_Earth]'] < 1.5]

In [41]:
# ---------------------------------------------------------------- #
#  Merge
# ---------------------------------------------------------------- #
# Perform left merges on various columns and combine results
merge_keys = ['HD Number 1', 'HD Number 2']
merged_HWO = pd.concat([df_HWO.merge(merged_df, left_on='hd_name', right_on=key, how='left') for key in merge_keys])

# Sort the combined DataFrame by 'source_id' to prioritize non-null values
merged_HWO.sort_values(by='source_id', ascending=False, inplace=True)

# Remove duplicate entries based on 'star_ID  ', keeping the first occurrence
merged_HWO.drop_duplicates(subset='hd_name', keep='first', inplace=True)

# Reset the index of the final DataFrame
merged_HWO.reset_index(drop=True, inplace=True)

# Save the final DataFrame to an Excel file
# filename = '../../results/merged_HWO.xlsx'
filename = '../../results/merged_HWO_Me<1.5.xlsx'
merged_HWO.to_excel(filename, index=False)
adjust_column_widths(filename)
